In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import DBSCAN
from itertools import product

In [ ]:
def generate_noise(width, length, amount, step, std=10, means=(1,5)):
        samples = np.zeros((width, length, amount, 2))

        clusters = np.random.randint(
            means[0], means[1] + 1, size=(width, length)
        )

        # calculate centers
        grid_width = (np.arange(width) + 1) * step
        grid_length = (np.arange(length) + 1) * step
        mean = np.array(
            [
                np.repeat(grid_width, len(grid_length)),
                np.tile(grid_length, len(grid_width)),
            ]
        ).T
        noise = np.random.randn(means[1], width * length, 2) * std
        centers = (noise + mean).reshape((means[1], width, length, 2))

        for i in range(width):
            for j in range(length):
                samples[i, j, :] = make_blobs(
                    n_samples=amount, centers=centers[0 : clusters[i, j], i, j, :]
                )[0]

        return samples, (grid_width, grid_length)

In [ ]:
np.random.seed(0)
data, map_grid = generate_noise(3, 3, 50, 10)

In [ ]:
plt.plot(data[0,0,:,0], data[0,0,:,1], 'o') # example of 5 clusters in position 0,0
plt.show()

In [ ]:
def generate_map(noise, eps=2, min_samples=3):
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(noise)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    return labels, core_samples_mask, n_clusters_

In [ ]:
def plot_clusters(X, labels, core_sapmles_mask, n_clusters_):
    
    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each)
              for each in np.linspace(0, 1, len(unique_labels))]
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]

        class_member_mask = (labels == k)

        xy = X[class_member_mask & core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=14)

        xy = X[class_member_mask & ~core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=6)
    plt.title('Estimated number of clusters: %d' % n_clusters_)

In [ ]:
labels = np.zeros((3, 3, 50), dtype=int)
for x,y in product(range(3), range(3)):
    labels[x,y,:], core_samples_mask, n_clusters_ = generate_map(data[x,y,:,:])
    plot_clusters(data[x,y,:,:], labels[x,y,:], core_samples_mask, n_clusters_)
    plt.show()

In [ ]:
# estimate parameters
params = [[[] for i in range(3)] for i in range(3)]
for x,y in product(range(3), range(3)):
    for i in range(np.max(labels[x,y,:]) + 1):
        mask = labels[x,y] == i
        mean_noise = data[x,y,mask,:].mean(axis=0) - np.array([(x+1) * 10,(y+1) * 10])
        cov_noise = np.cov(data[x,y,mask,:].T)
        weight = mask.sum() / 50
        params[x][y].append((mean_noise, cov_noise, weight))

print(params)

In [ ]:
# dynamics model

walk = []
start_state = np.array([20, 20, 0, 0], dtype=float)
walk.append(start_state)

def transition_function(current_state, x_range=(10, 40), y_range=(10, 40), std=1):
    """Performs a one step transition assuming sensing interval of one
    
    Format of current_state = [x,y,x',y']
    """
    next_state = np.copy(current_state)
    next_state[0:2] += current_state[2:4]
    next_state[2:4] += np.random.randn(2) * std
    
    next_state[0] = np.clip(next_state[0], x_range[0], x_range[1])
    next_state[1] = np.clip(next_state[1], y_range[0], y_range[1])
    return next_state

next_state = transition_function(start_state)
walk.append(next_state)
for i in range(100):
    next_state = transition_function(next_state)
    walk.append(next_state)
walk = np.array(walk)
walk.shape
plt.plot(walk[:,0], walk[:, 1])
plt.show()

In [ ]:
# measurement noise map augmented particle filter

def find_nearest_map_position(x,y, map_grid):
    x_pos = np.searchsorted(map_grid[0], x, side="right")
    y_pos = np.searchsorted(map_grid[1], y, side="right")
    print(x_pos)
    x_valid = (x_pos != 0) & (x_pos < len(map_grid[0]))
    y_valid = (y_pos != 0) & (y_pos < len(map_grid[1]))
        
    x_dist_left = np.abs(map_grid[0][x_valid] - x[x >= np.min(map_grid[0])])
    x_dist_left = np.abs(x[x >= np.min(map_grid[0])] - map_grid[0][x_idxs - 1])
    y_dist_right = np.abs(map_grid[1][y_idxs] - y[y >= np.min(map_grid[1])])
    y_dist_left = np.abs(y[y >= np.min(map_grid[1])] - map_grid[1][x_idxs - 1])

    (x_pos[x_idxs])[x_dist_left < x_dist_right] += 1
    (y_pos[y_idxs])[y_dist_left < y_dist_right] += 1
    
    x_pos = np.clip(x_pos, 0, len(map_grid[0]) - 1)
    y_pos = np.clip(x_pos, 0, len(map_grid[1]) - 1)
    
    return x_pos, y_pos

def reweight_samples(x,y,w):
    x_pos, y_pos = find_nearest_map_position(x,y)
    

print(find_nearest_map_position(
    np.array([-1, 31, 29, 11]),
    np.array([-1, 31, 29, 11]),
    map_grid
))
print(map_grid)